In [9]:
import simpy
import random
import pandas as pd
import networkx as nx

# Constants for conversions and simulation parameters
KB_TO_MB = 1 / 1024
BIT_TO_MBIT = 1 / 1024
MHZ_TO_HZ = 1e6
ENV_RUN_TIME = 1000

transmission_bandwidth = 1 * MHZ_TO_HZ  # in Hz
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # in Mbit
max_power_consumption = 1000  # in Joules
local_device_cycle_freq = random.uniform(2, 4) * 1e9  # in Hz
mec_server_cycle_freq = random.uniform(4, 8) * 1e9  # in Hz
local_device_power = random.uniform(100, 400) / 1000  # in Watts
clock_cycles_per_bit = random.randint(800, 1200)

L_c = 10**-24   # CPU power usage coefficient, adjust as needed

class MECServer:
    def __init__(self, env, capacity):
        self.env = env
        self.capacity = capacity  # GHz, converted to Hz for calculations
        self.processor = simpy.Resource(env, capacity=int(capacity))
        self.dag = nx.DiGraph()

    def add_task_to_dag(self, task_id, dependencies, task_data):
        self.dag.add_node(task_id, data=task_data, vk=random.random())
        for dep in dependencies:
            self.dag.add_edge(dep, task_id)

    def process_task(self, task_id):
        task_info = self.dag.nodes[task_id]
        cycles_required = task_info['data'] * BIT_TO_MBIT * 1e6 * clock_cycles_per_bit
        local_time = cycles_required / local_device_cycle_freq
        edge_time = cycles_required / self.capacity
        total_task_time = (1 - task_info['vk']) * local_time + task_info['vk'] * edge_time
        
        yield self.env.timeout(total_task_time)
        return total_task_time

    def execute_tasks(self):
        for task in nx.topological_sort(self.dag):
            yield self.env.process(self.process_task(task))
            ct = self.env.now  # Completion time of the current task
            self.dag.nodes[task]['ct'] = ct
            
class Device:
    def __init__(self, env, mec_servers, logs, num_devices):
        self.env = env
        self.mec_servers = mec_servers
        self.logs = logs
        self.num_devices = num_devices
        self.y_nt = 198.5  # Weight factor for local time cost, adjustable
        self.y_ne = 63.5  # Weight factor for local energy cost, adjustable

    def send_task(self, task_data, dependencies=[]):
        start_time = self.env.now
        H_n = random.randint(1, 10)  # Randomly decide the number of tasks not offloaded
        R_n = random.random()  # Random offloading ratio

        # Local computation time and energy based on the model
        RT = H_n * (1 - R_n) / local_device_cycle_freq
        AT = L_c * (local_device_cycle_freq ** 2) * H_n * (1 - R_n)
        DT = self.y_nt * RT + self.y_ne * AT  # Weighted waste including time and energy factors

        # Log local computation
        self.logs.append({
            'num_devices': self.num_devices,
            'task_data': task_data,
            'local_completion_time': RT,
            'energy_consumed': AT,
            'total_weighted_waste': DT
        })

        # Assuming the task might still be offloaded
        mec = min(self.mec_servers, key=lambda x: len(x.dag))
        task_id = start_time
        mec.add_task_to_dag(task_id, dependencies, task_data)

        data_bits = task_data * BIT_TO_MBIT * 1e6
        transmission_time = data_bits / transmission_bandwidth
        energy_consumed = transmission_time * local_device_power

        yield self.env.timeout(transmission_time)
        yield self.env.process(mec.execute_tasks())

        ct = mec.dag.nodes[task_id]['ct']
        total_energy = energy_consumed + (ct - start_time) * local_device_power

        self.logs.append({
            'num_devices': self.num_devices,
            'task_data': task_data,
            'completion_time': ct,
            'energy_consumed': total_energy
        })

# Initialize environment
env = simpy.Environment()
mecs = [MECServer(env, mec_server_cycle_freq) for _ in range(num_devices_list[-1])]
logs = []

# Setup devices and start simulation
for num_devices in num_devices_list:
    devices = [Device(env, mecs, logs, num_devices) for _ in range(num_devices)]
    for device in devices:
        task_size = random.choice(task_data)
        env.process(device.send_task(task_size))

env.run(until=ENV_RUN_TIME)

# Convert logs to DataFrame and print
df_logs = pd.DataFrame(logs)
print(df_logs.head())
#df_logs.to_csv('metric_log2.csv')
if not df_logs.empty and df_logs['energy_consumed'].max() > max_power_consumption:
    print("Warning: Power consumption exceeded the maximum limit.")


   num_devices  task_data  local_completion_time  energy_consumed  \
0           50         15           4.526992e-10         0.000012   
1           50         45           6.174416e-10         0.000016   
2           50         10           2.905381e-09         0.000075   
3           50         25           1.770117e-09         0.000046   
4           50         40           1.985425e-10         0.000005   

   total_weighted_waste  completion_time  
0              0.000740              NaN  
1              0.001010              NaN  
2              0.004751              NaN  
3              0.002895              NaN  
4              0.000325              NaN  
